In [2]:
@time for i in 1:10 
sleep(2)
end 

 20.102833 seconds (617 allocations: 18.125 KiB)


In [3]:
@time for i in 1:10 
     @async sleep(2)
end 

  0.007297 seconds (6.23 k allocations: 370.582 KiB)


Notice that by adding @async we have reduced the time. We have added threads which do the calculations parallely and hence time required is very less.

In [5]:
?@async

```
@async
```

Wrap an expression in a [`Task`](@ref) and add it to the local machine's scheduler queue.

Values can be interpolated into `@async` via `$`, which copies the value directly into the constructed underlying closure. This allows you to insert the *value* of a variable, isolating the aysnchronous code from changes to the variable's value in the current task.

!!! compat "Julia 1.4"
    Interpolating values via `$` is available as of Julia 1.4.



# @async and @sync
Refer  [Site](https://riptutorial.com/julia-lang/example/15919/-async-and--sync)
* According to the documentation under ?@async, "@async wraps an expression in a Task." What this means is that for whatever falls within its scope, Julia will start this task running but then proceed to whatever comes next in the script without waiting for the task to complete. Thus, for instance, without the macro you will get:


In [1]:
@time sleep(2)

  2.009118 seconds (64 allocations: 1.984 KiB)


But with the macro, you get:

In [6]:
@time @async sleep(2)


  0.000066 seconds (26 allocations: 2.250 KiB)


7

Julia thus allows the script to proceed (and the @time macro to fully execute) without waiting for the task (in this case, sleeping for two seconds) to complete.


The @sync macro, by contrast, will "Wait until all dynamically-enclosed uses of @async, @spawn, @spawnat and @parallel are complete." (according to the documentation under ?@sync). Thus, we see:

In [9]:
@time @sync @async sleep(2)

  2.045602 seconds (1.34 k allocations: 70.828 KiB)


Task (done) @0x000000003b4a2b30

In this simple example then, there is no point to including a single instance of @async and @sync together. But, where @sync can be useful is where you have @async applied to multiple operations that you wish to allow to all start at once without waiting for each to complete.

For example:

In [10]:
@time @sync @time for i in 1:10
    @async sleep(2)
    end

  0.000058 seconds (58 allocations: 8.734 KiB)
  2.110324 seconds (1.65 k allocations: 93.312 KiB)


So 0.000058 seconds is the time required to generate the threads and so all operations now are run **parallely** and hence take ~2 seconds.

Notice that the above is actually **concurrency** since there is no computation involved.

## Examples of the Differences
Synchronous = Thread will complete an action Blocking = Thread will wait until action is completed

* Asynchronous + Non-Blocking: I/O

* Asynchronous + Blocking: Threaded atomics (demonstrated next lecture)

* Synchronous + Blocking: Standard computing, @sync

* Synchronous + Non-Blocking: Webservers where an I/O operation can be performed, but one never checks if the operation is completed.

# Multithreading
If your threads are independent, then it may make sense to run them in parallel. This is the form of parallelism known as multithreading. To understand the data that is available in a multithreaded setup, let's look at the picture of threads again:
![img](https://blog-assets.risingstack.com/2017/02/kernel-processes-and-threads-1.png)


Each thread has its own call stack, but it's the process that holds the heap. This means that dynamically-sized heap allocated objects are shared between threads with no cost, a setup known as shared-memory computing.

# Loop-Based Multithreading with @threads
Let's look back at our Lorenz dynamical system from before:

In [11]:
using StaticArrays, BenchmarkTools
function lorenz(u,p)
  α,σ,ρ,β = p
  @inbounds begin
    du1 = u[1] + α*(σ*(u[2]-u[1]))
    du2 = u[2] + α*(u[1]*(ρ-u[3]) - u[2])
    du3 = u[3] + α*(u[1]*u[2] - β*u[3])
  end
  @SVector [du1,du2,du3]
end
function solve_system_save!(u,f,u0,p,n)
  @inbounds u[1] = u0
  @inbounds for i in 1:length(u)-1
    u[i+1] = f(u[i],p)
  end
  u
end
p = (0.02,10.0,28.0,8/3)
u = Vector{typeof(@SVector([1.0,0.0,0.0]))}(undef,1000)
@btime solve_system_save!(u,lorenz,@SVector([1.0,0.0,0.0]),p,1000)

  4.700 μs (0 allocations: 0 bytes)


1000-element Array{SArray{Tuple{3},Float64,1,3},1}:
 [1.0, 0.0, 0.0]
 [0.8, 0.56, 0.0]
 [0.752, 0.9968000000000001, 0.008960000000000001]
 [0.80096, 1.3978492416000001, 0.023474005333333336]
 [0.92033784832, 1.8180538219817644, 0.04461448495326095]
 [1.099881043052353, 2.296260732619613, 0.07569952060880669]
 [1.339156980965805, 2.864603692722823, 0.12217448583728006]
 [1.6442463233172087, 3.5539673118971193, 0.19238159391549564]
 [2.026190521033191, 4.397339452147425, 0.2989931959555302]
 [2.5004203072560376, 5.431943011293093, 0.4612438424853632]
 [3.0867248480634486, 6.700473453723668, 0.7082869831520391]
 [3.8094745691954923, 8.25130415895562, 1.0841620354518975]
 [4.697840487147518, 10.136982080467158, 1.655002727352565]
 ⋮
 [10.49730559336705, 4.660822889495989, 35.336831929448614]
 [9.330009052592839, 3.0272670946941713, 34.430722536963344]
 [8.069460661013105, 1.766747763108672, 33.159305922954225]
 [6.8089180814322185, 0.8987564841782779, 31.6759436385101]
 [5.6268857619814305

In order to use multithreading on this code, we need to take a look at the dependency graph and see what items can be calculated independently of each other. Notice that

```math
σ*(u[2]-u[1])
ρ-u[3]
u[1]*u[2]
β*u[3]
```
are all independent operations, so in theory we could split those off to different threads, move up, etc.

Or we can have three threads:

```math 
u[1] + α*(σ*(u[2]-u[1]))
u[2] + α*(u[1]*(ρ-u[3]) - u[2])
u[3] + α*(u[1]*u[2] - β*u[3])
```
all don't depend on the output of each other, so these tasks can be run in parallel. We can do this by using Julia's Threads.@threads macro which puts each of the computations of a loop in a different thread. The threaded loops do not allow you to return a value, so how do you build up the values for the @SVector?


It's not possible!


There is a shared heap, but the stacks are thread local. This means that a value cannot be stack allocated in one thread and magically appear when re-entering the main thread: it needs to go on the heap somewhere. But if it needs to go onto the heap, then it makes sense for us to have preallocated its location. But if we want to preallocate $du[1], du[2], and du[3]$, then it makes sense to use the fully non-allocating update form:

In [20]:
function lorenz!(du,u,p)
  α,σ,ρ,β = p
  @inbounds begin
    du[1] = u[1] + α*(σ*(u[2]-u[1]))
    du[2] = u[2] + α*(u[1]*(ρ-u[3]) - u[2])
    du[3] = u[3] + α*(u[1]*u[2] - β*u[3])
  end
end
function solve_system_save_iip!(u,f,u0,p,n)
  @inbounds u[1] = u0
  @inbounds for i in 1:length(u)-1
    f(u[i+1],u[i],p)
  end
  u
end
p = (0.02,10.0,28.0,8/3)
u = [Vector{Float64}(undef,3) for i in 1:1000]
@btime solve_system_save_iip!(u,lorenz!,[1.0,0.0,0.0],p,1000)

  5.960 μs (1 allocation: 112 bytes)


1000-element Array{Array{Float64,1},1}:
 [1.0, 0.0, 0.0]
 [0.8, 0.56, 0.0]
 [0.752, 0.9968000000000001, 0.008960000000000001]
 [0.80096, 1.3978492416000001, 0.023474005333333336]
 [0.92033784832, 1.8180538219817644, 0.04461448495326095]
 [1.099881043052353, 2.296260732619613, 0.07569952060880669]
 [1.339156980965805, 2.864603692722823, 0.12217448583728006]
 [1.6442463233172087, 3.5539673118971193, 0.19238159391549564]
 [2.026190521033191, 4.397339452147425, 0.2989931959555302]
 [2.5004203072560376, 5.431943011293093, 0.4612438424853632]
 [3.0867248480634486, 6.700473453723668, 0.7082869831520391]
 [3.8094745691954923, 8.25130415895562, 1.0841620354518975]
 [4.697840487147518, 10.136982080467158, 1.655002727352565]
 ⋮
 [10.49730559336705, 4.660822889495989, 35.336831929448614]
 [9.330009052592839, 3.0272670946941713, 34.430722536963344]
 [8.069460661013105, 1.766747763108672, 33.159305922954225]
 [6.8089180814322185, 0.8987564841782779, 31.6759436385101]
 [5.6268857619814305, 0.38019737

and now we multithread:

In [14]:
using Base.Threads
function lorenz_mt!(du,u,p)
  α,σ,ρ,β = p
  let du=du, u=u, p=p
    Threads.@threads for i in 1:3
      @inbounds begin
        if i == 1
          du[1] = u[1] + α*(σ*(u[2]-u[1]))
        elseif i == 2
          du[2] = u[2] + α*(u[1]*(ρ-u[3]) - u[2])
        else
          du[3] = u[3] + α*(u[1]*u[2] - β*u[3])
        end
        nothing
      end
    end
  end
  nothing
end
function solve_system_save_iip!(u,f,u0,p,n)
  @inbounds u[1] = u0
  @inbounds for i in 1:length(u)-1
    f(u[i+1],u[i],p)
  end
  u
end
p = (0.02,10.0,28.0,8/3)
u = [Vector{Float64}(undef,3) for i in 1:1000]
@btime solve_system_save_iip!(u,lorenz_mt!,[1.0,0.0,0.0],p,1000);

  1.924 ms (5995 allocations: 967.89 KiB)


**Parallelism doesn't always make things faster**
 There are two costs associated with this code. For one, we had to go to the slower heap+mutation version, so its implementation starting point is slower. But secondly, and more importantly, the cost of spinning a new thread is non-negligable. In fact, here we can see that it even needs to make a small allocation for the new context. The total cost is on the order of It's on the order of 50ns: not huge, but something to take note of. So what we've done is taken almost free calculations and made them ~50ns by making each in a different thread, instead of just having it be one thread with one call stack.

The moral of the story is that you need to make sure that there's enough work per thread in order to effectively accelerate a program with parallelism.

## Data-Parallel Problems
So not every setup is amenable to parallelism. Dynamical systems are natorious for being quite difficult to parallelize because the dependency of the future time step on the previous time step is clear, meaning that one cannot easily "parallelize through time" (though it is possible, which we will study later).

However, one common way that these systems are generally parallelized is in their inputs. The following questions allow for independent simulations:

What steady state does an input u0 go to for some list/region of initial conditions?

How does the solution very when I use a different p?

The problem has a few descriptions. For one, it's called an embaressingly parallel problem since the problem can remain largely intact to solve the parallelism problem. To solve this, we can use the exact same solve_system_save_iip!, and just change how we are calling it. Secondly, this is called a data parallel problem, since it parallelized by splitting up the input data (here, the possible u0 or ps) and acting on them independently.

In [18]:
u = Vector{typeof(@SVector([1.0,0.0,0.0]))}(undef,1000)

1000-element Array{SArray{Tuple{3},Float64,1,3},1}:
 [6.4e-323, 5.152288944e-315, 2.27e-322]
 [5.45569341e-315, NaN, 0.0]
 [0.0, -1.022081779e-314, 7.15014174e-316]
 [0.0, 8.4e-323, 4.642775625e-315]
 [0.0, 7.1501449e-316, 0.0]
 [8.4e-323, 4.642775625e-315, 5.0e-324]
 [4.77155874e-316, 0.0, 8.4e-323]
 [4.642775625e-315, 1.0e-323, 7.15014174e-316]
 [0.0, 8.4e-323, 4.642775625e-315]
 [1.5e-323, 0.0, -5.442527131007e-312]
 [0.0, 0.0, 4.649742074e-315]
 [0.0, 0.0, 4.176818213e-315]
 [0.0, 4.201675595e-315, 7.2111091e-316]
 ⋮
 [NaN, 2.1219957905e-314, 0.0]
 [0.0, 0.0, 0.0]
 [0.0, 5.455810763e-315, 8.487983164e-314]
 [0.0, 0.0, 0.0]
 [0.0, 0.0, NaN]
 [2.1219957905e-314, 0.0, 0.0]
 [0.0, 0.0, 0.0]
 [5.455811316e-315, 8.487983164e-314, 0.0]
 [0.0, 0.0, 0.0]
 [0.0, NaN, 2.1219957905e-314]
 [0.0, 0.0, 0.0]
 [0.0, 0.0, 5.45581187e-315]

# Multi-Threading


## Starting Julia with multiple threads

By default, Julia starts up with a single thread of execution. This can be verified by using the
command [`Threads.nthreads()`](@ref):

```julia-repl
julia> Threads.nthreads()
1
```

The number of execution threads is controlled either by using the
`-t`/`--threads` command line argument or by using the
[`JULIA_NUM_THREADS`](@ref JULIA_NUM_THREADS) environment variable. When both are
specified, then `-t`/`--threads` takes precedence.

!!! compat "Julia 1.5"
    The `-t`/`--threads` command line argument requires at least Julia 1.5.
    In older versions you must use the environment variable instead.

Lets start Julia with 4 threads:

```bash
$ julia --threads 4
```

Let's verify there are 4 threads at our disposal.

```julia-repl
julia> Threads.nthreads()
4
```

But we are currently on the master thread. To check, we use the function [`Threads.threadid`](@ref)

```julia-repl
julia> Threads.threadid()
1
```



## The `@threads` Macro

Let's work a simple example using our native threads. Let us create an array of zeros:

```jldoctest
julia> a = zeros(10)
10-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
```

Let us operate on this array simultaneously using 4 threads. We'll have each thread write its
thread ID into each location.

Julia supports parallel loops using the [`Threads.@threads`](@ref) macro. This macro is affixed
in front of a `for` loop to indicate to Julia that the loop is a multi-threaded region:

```julia-repl
julia> Threads.@threads for i = 1:10
           a[i] = Threads.threadid()
       end
```

The iteration space is split among the threads, after which each thread writes its thread ID
to its assigned locations:

```julia-repl
julia> a
10-element Array{Float64,1}:
 1.0
 1.0
 1.0
 2.0
 2.0
 2.0
 3.0
 3.0
 4.0
 4.0
```

Note that [`Threads.@threads`](@ref) does not have an optional reduction parameter like [`@distributed`](@ref).


In [21]:
using Statistics
function compute_trajectory_mean(u0,p)
  u = Vector{typeof(@SVector([1.0,0.0,0.0]))}(undef,1000)
  solve_system_save!(u,lorenz,u0,p,1000);
  mean(u)
end
@btime compute_trajectory_mean(@SVector([1.0,0.0,0.0]),p)

  5.840 μs (3 allocations: 23.55 KiB)


3-element SArray{Tuple{3},Float64,1,3} with indices SOneTo(3):
 -0.3114996234648468
 -0.30974901748976497
 26.02460355858298

We can make this faster by preallocating the cache vector u. For example, we can globalize it:


In [22]:
u = Vector{typeof(@SVector([1.0,0.0,0.0]))}(undef,1000)
function compute_trajectory_mean2(u0,p)
  # u is automatically captured
  solve_system_save!(u,lorenz,u0,p,1000);
  mean(u)
end
@btime compute_trajectory_mean2(@SVector([1.0,0.0,0.0]),p)

  5.716 μs (3 allocations: 112 bytes)


3-element SArray{Tuple{3},Float64,1,3} with indices SOneTo(3):
 -0.3114996234648468
 -0.30974901748976497
 26.02460355858298

But this is still allocating? The issue with this code is that u is a global, and captured globals cannot be inferred because their type can change at any time. Thus what we can do instead is capture a constant:

In [23]:
const _u_cache = Vector{typeof(@SVector([1.0,0.0,0.0]))}(undef,1000)
function compute_trajectory_mean3(u0,p)
  # u is automatically captured
  solve_system_save!(_u_cache,lorenz,u0,p,1000);
  mean(_u_cache)
end
@btime compute_trajectory_mean3(@SVector([1.0,0.0,0.0]),p)

3-element SArray{Tuple{3},Float64,1,3} with indices SOneTo(3):
 -0.3114996234648468
 -0.30974901748976497
 26.02460355858298

  5.516 μs (1 allocation: 32 bytes)


Now it's just allocating the output. The other way to do this is to use a closure which encapsolates the cache data:

In [24]:
function _compute_trajectory_mean4(u,u0,p)
  solve_system_save!(u,lorenz,u0,p,1000);
  mean(u)
end
compute_trajectory_mean4(u0,p) = _compute_trajectory_mean4(_u_cache,u0,p)
@btime compute_trajectory_mean4(@SVector([1.0,0.0,0.0]),p)

  5.667 μs (1 allocation: 32 bytes)


3-element SArray{Tuple{3},Float64,1,3} with indices SOneTo(3):
 -0.3114996234648468
 -0.30974901748976497
 26.02460355858298

This is the same, but a bit more explicit. Now let's create our parameter search function. Let's take a sample of parameters:


In [25]:
ps = [(0.02,10.0,28.0,8/3) .* (1.0,rand(3)...) for i in 1:1000]

1000-element Array{NTuple{4,Float64},1}:
 (0.02, 0.44123529418802043, 1.6851587809641204, 0.239660080649843)
 (0.02, 1.4539007670432236, 24.306366989092673, 0.06084003629815922)
 (0.02, 2.342060697202677, 6.294287410587492, 0.3969872706554588)
 (0.02, 7.85637985022791, 1.2384312840302396, 0.09535698102462338)
 (0.02, 2.278338963700901, 0.04666812880234872, 1.3250117194867337)
 (0.02, 6.341917254884553, 3.916239784877317, 2.4020416363006896)
 (0.02, 1.2120950017638132, 15.75021141818561, 2.0231803103046486)
 (0.02, 3.877332504082387, 1.329689384889713, 0.001606167319778843)
 (0.02, 6.58345567263966, 21.48182064680297, 0.5816402364282525)
 (0.02, 3.54592397066795, 19.90892687068405, 1.9555446213006378)
 (0.02, 5.457010216458595, 27.227377982882814, 1.986277023603229)
 (0.02, 1.5538726752633925, 7.538595664814308, 2.0818425591152563)
 (0.02, 3.7258647369503572, 5.41245585154526, 1.7688656622098287)
 ⋮
 (0.02, 9.748810821891096, 0.5311018319797558, 0.9232324429563825)
 (0.02, 7.77999080387

And let's get the mean of the trajectory for each of the parameters.

In [26]:
serial_out = map(p -> compute_trajectory_mean4(@SVector([1.0,0.0,0.0]),p),ps)

1000-element Array{SArray{Tuple{3},Float64,1,3},1}:
 [0.47455120703230896, 0.4082570562046704, 0.7547635359056643]
 [0.33288576370319045, 0.29341270501833583, 25.053836794944047]
 [-0.9345130840839617, -0.9829389438577304, 5.092775704167578]
 [0.18559742417319422, 0.1798584953244467, 0.2541354765002672]
 [0.023001977464101313, 0.0010561648460699123, 0.00012118356439335954]
 [2.5611856821075842, 2.5741682046451753, 2.7562618467592994]
 [5.334207907584775, 5.518302329947609, 14.259905241170204]
 [0.1446332130025358, 0.13176477904169734, 0.6087904362825064]
 [0.2887811815163382, 0.274814425567736, 20.26996712736188]
 [-0.05688544989742323, 0.05668704016694275, 15.536350180043843]
 [-0.12434591786917532, -0.0037302214036733796, 24.0773676260771]
 [3.5943963521926494, 3.680937939931901, 6.273662851118569]
 [2.7205663508773212, 2.74462694713363, 4.208600644486312]
 ⋮
 [0.010914306110518922, 0.00578551420678347, 0.0002792584024665549]
 [0.662545216373358, 0.6597086434905194, 1.797448186504609